In [ ]:
import logging
import ngsolve as ngs
from ngsolve.meshes import MakeQuadMesh
import numpy as np

import regpy.stoprules as rules
from regpy.operators.ngsolve import Coefficient
from regpy.solvers import HilbertSpaceSetting
from regpy.solvers.landweber import Landweber
from regpy.hilbert import L2, Sobolev
from regpy.discrs.ngsolve import NgsSpace

In [ ]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s %(levelname)s %(name)-40s :: %(message)s'
)

In [ ]:
meshsize_domain = 10
meshsize_codomain = 10

mesh = MakeQuadMesh(meshsize_domain)
fes_domain = ngs.L2(mesh, order=2)
domain = NgsSpace(fes_domain)

mesh = MakeQuadMesh(meshsize_codomain)
fes_codomain = ngs.H1(mesh, order=3, dirichlet="left|top|right|bottom")
codomain = NgsSpace(fes_codomain)

In [ ]:
rhs = 10 * ngs.sin(ngs.x) * ngs.sin(ngs.y)
op = Coefficient(
    domain, rhs, codomain=codomain, bc_left=0, bc_right=0, bc_bottom=0, bc_top=0, diffusion=False,
    reaction=True, dim=2
)

In [ ]:
exact_solution_coeff = ngs.x + 1
gfu_exact_solution = ngs.GridFunction(op.fes_domain)
gfu_exact_solution.Set(exact_solution_coeff)
exact_solution = gfu_exact_solution.vec.FV().NumPy()
exact_data = op(exact_solution)

In [ ]:
fes_noise=ngs.L2(fes_codomain.mesh, order=1)
gfu_noise_order1=ngs.GridFunction(fes_noise)
gfu_noise_order1.vec.FV().NumPy()[:]=0.0001*np.random.randn(fes_noise.ndof)
gfu_noise=ngs.GridFunction(fes_codomain)
gfu_noise.Set(gfu_noise_order1)
noise=gfu_noise.vec.FV().NumPy()

In [ ]:
data = exact_data+noise

In [ ]:
init = 1 + ngs.x + 5*ngs.x*(1-ngs.x)*ngs.y*(1-ngs.y)

init_gfu = ngs.GridFunction(op.fes_domain)
init_gfu.Set(init)
init_solution = init_gfu.vec.FV().NumPy().copy()
init_data = op(init_solution)

In [ ]:
setting = HilbertSpaceSetting(op=op, Hdomain=L2, Hcodomain=Sobolev)

In [ ]:
landweber = Landweber(setting, data, init_solution, stepsize=1)
stoprule = (
        rules.CountIterations(1000) +
        rules.Discrepancy(setting.Hcodomain.norm, data, noiselevel=setting.Hcodomain.norm(noise), tau=1.1))

In [ ]:
reco, reco_data = landweber.run(stoprule)

In [ ]:
gfu_reco = ngs.GridFunction(op.fes_domain)
gfu_data = ngs.GridFunction(op.fes_codomain)
gfu_reco_data = ngs.GridFunction(op.fes_codomain)

gfu_reco.vec.FV().NumPy()[:] = reco
gfu_data.vec.FV().NumPy()[:] = data
gfu_reco_data.vec.FV().NumPy()[:] = reco_data

In [ ]:
error = exact_solution_coeff-gfu_reco
vtkout = ngs.VTKOutput(mesh, coefs=[exact_solution_coeff, init, gfu_reco, gfu_data, gfu_reco_data, error], 
                       names =["exact","init","reco","data","reco_data","error_reco"], 
                       filename = "reac_coef", subdivision=2)
vtkout.Do()

In [ ]:
#%matplotlib inline
#from pyvista import set_plot_theme
#set_plot_theme('document')

In [ ]:
import pyvista as pv
plot_mesh = pv.read('reac_coef.vtk')
plot_mesh2 = plot_mesh.copy()


In [ ]:
p = pv.Plotter()
plot_mesh_reco= plot_mesh2.warp_by_scalar("reco",normal=(0,0,1),factor=1)
p.add_mesh(plot_mesh, scalars="reco")
p.show(use_panel=False)


In [ ]:
p = pv.Plotter()
plot_mesh_exact = plot_mesh.warp_by_scalar("exact",normal=(0,0,1),factor=1)
p.add_mesh(plot_mesh, scalars="exact") 
p.show(use_panel=False)

In [ ]:
p = pv.Plotter()
plot_mesh_error = plot_mesh.warp_by_scalar("error_reco",normal=(0,0,1),factor=1)
p.add_mesh(plot_mesh_error, scalars="error_reco") 
p.show(use_panel=False)